In [ ]:
# !pip install langchain-text-splitters qdrant-client langchain-qdrant sentence-transformers torch 
# !pip install ragas rapidfuzz ipywidgets langchain-huggingface accelerate

In [1]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import torch,uuid
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFaceEmbeddings ,HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.language_models.llms import LLM
from typing import Any, List, Optional

In [2]:
### 파일 data load

# 파일 경로 지정
file_path = '사회복지_법령_전체.txt'

# 파일 내용이 담긴 변수
law_data=''


# 파일 내용 load
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        law_data = f.read()
    print(f"전체 글자 수: {len(law_data):,}자")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다.")

전체 글자 수: 4,987,798자


In [3]:
### 문서 분할

# 필요없는 부분 법령명으로 replace
law_data = law_data.replace("""판례
연혁
위임행정규칙
규제
생활법령
한눈보기""","")

# 분할할 방식 설정
text_splitter = CharacterTextSplitter(
    separator="\n\n\n",
    chunk_size=1,           # 구분자 기준으로 바로 쪼개지도록 최소값 설정
    chunk_overlap=0,        # 중복 없음
    is_separator_regex=False # 일반 문자열로 취급
)

# 분할
chunks = text_splitter.split_text(law_data)

# vectorDB에 넣을 문서 리스트
documents=[]

# vectorDB에 넣을 형식으로 변환
for chunk in chunks:
    #문서의 법령을 제목으로 사용하기 위한 개행으로 split
    law_name = chunk.splitlines()
    
    # vectorDB에 넣을 형식으로 변환
    doc = Document(
        page_content=chunk,
        metadata={
            "law_name": law_name[0], 
            "length": len(chunk)
        }
    )

    # vectorDB에 넣을 list에 변환한 문서 append
    documents.append(doc)


Created a chunk of size 1764, which is longer than the specified 1
Created a chunk of size 329, which is longer than the specified 1
Created a chunk of size 1064, which is longer than the specified 1
Created a chunk of size 1089, which is longer than the specified 1
Created a chunk of size 1017, which is longer than the specified 1
Created a chunk of size 2752, which is longer than the specified 1
Created a chunk of size 2117, which is longer than the specified 1
Created a chunk of size 449, which is longer than the specified 1
Created a chunk of size 459, which is longer than the specified 1
Created a chunk of size 2846, which is longer than the specified 1
Created a chunk of size 1244, which is longer than the specified 1
Created a chunk of size 1147, which is longer than the specified 1
Created a chunk of size 3147, which is longer than the specified 1
Created a chunk of size 4551, which is longer than the specified 1
Created a chunk of size 8678, which is longer than the specified 

In [4]:
# huggingface login
import os 
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()
login(os.getenv('HUGGINGFACE_API_KEY'))

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
### VectorDB에 저장

# 모델에 따라 달라질 코드(임베딩)
embeddings = HuggingFaceEmbeddings(
    model_name="dragonkue/multilingual-e5-small-ko-v2",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True} # 의미 기반 검색 최적화
)

# qdrant 연결
url = "http://localhost:6333"
client = QdrantClient(url=url)

# 각 법령의 구분 키값
ids = [
    str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{doc.metadata['law_name']}_{i}")) 
    for i, doc in enumerate(documents)
]

# DB명
collection_name = "B-TEAM"

# vectorDB에 저장
vector_store = QdrantVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    ids = ids,
    url=url,
    collection_name=collection_name,
)

In [ ]:
### RAG


# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="dragonkue/multilingual-e5-small-ko-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# vectorDB연결
url = "http://localhost:6333"
collection_name = "B-TEAM"

vector_store = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name=collection_name,
    url=url,
)

# 질문에 대한 답은 가장 유사한 것 하나
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

#llm 모델 설정
model_id = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

torch_dtype = torch.float16 if device == "cuda" else torch.float32
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto",               
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True
)

model_engine = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=model_engine)

# 프롬프트 구성
qa_system_prompt = """당신은 법률 전문 어시스턴트입니다. 
제공된 [법령 정보] 중 질문과 가장 일치하는 단 하나의 근거만을 바탕으로 명확하게 답변하세요. 
정보가 부족하면 모른다고 답하고, 답변은 한국어로 하세요.

[법령 정보]:
{context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# LCEL 체인
def extract_content(docs):
    return docs[0].page_content if docs else "관련 법령 없음"

def extract_content(docs):
    return docs[0].page_content if docs else "관련 법령 없음"

# LCEL 체인: 구조 변경 없이 그대로 사용
rag_chain = (
    {
        # x["input"]을 통해 딕셔너리에서 순수 질문 텍스트만 꺼내 retriever에게 전달
        "context": (lambda x: x["input"]) | retriever | extract_content,
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"]
    }
    | qa_prompt 
    | llm  # 이전에 만든 CustomLLM 또는 FinalLawLLM
)

#실제 대화
chat_history = []
print("법률 상담을 시작합니다. (종료: exit)")

while True:
    user_input = input("\n나: ")
    if user_input.lower() in ["exit", "종료"]: break

    # 체인 호출
    response = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
    print(f"Qwen: {response}")

    # 대화 기록 업데이트 (최근 3턴만 유지하여 CPU 부담 감소)
    chat_history.extend([HumanMessage(content=user_input), AIMessage(content=response)])
    chat_history = chat_history[-6:]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu


법률 상담을 시작합니다. (종료: exit)
Qwen:  

답변:

사회복지사의 처우 개선은 보건복지부와 지방자치단체가 사회복지사의 처우를 개선하고 인권 및 복지를 증진함과 동시에 그 지위를 향상시키고, 사회복지사 등을 폭력으로부터 보호하기 위해 적극적인 노력을 하는데 있습니다. 또한, 보건복지부장관은 사회복지사 등의 보수에 관한 지침을 마련하고, 이를 지방자치단체에 준수하도록 요구하며, 사회복지사 등의 근로여건, 보수수준 및 지급실태, 인권침해 실태 및 그에 대한 조치 현황, 지침의 준수율 등에 대해 3년마다 조사·공표해야 합니다. 사회복지사 등은 사회복지법인 등의 운영과 관련된 위법ㆍ부당 행위 및 그 밖의 비리 사실 등을 관계 행정기관과 수사기관에 신고하는 행위로 인하여 징계 조치 등 신분상 불이익이나 근무조건상 차별을 받지 않도록 돕습니다. 또한, 사회복지사 등의 고용 안정을 위해 필요한 시책을 수립하고, 그 추진에 필요한 행정적ㆍ재정적 지원방안을 마련해야 합니다. 사회복지사 등의 근로조건과 관련하여 이 법 또는 다른 법률에 특별한 규정이 없는 경우 「근로기준법」에서 정하는 바에 따라 적용됩니다. 보건복지부장관은 제4항에 따른 조사 결과를 사회복지사 등의 처우개선 및 인권침해 예방을 위한 정책수립의 기초자료로 활용해야 합니다. 제4항에 따른 조사ㆍ공표의 방법과 내용 등에 필요한 사항은 대통령령으로 정합니다. 제4항에 따른 조사ㆍ공표의 방법과 내용 등에 필요한 사항은 대통령령으로 정합니다. 사회복지사 등의 권익을 보호하기 위한 권익지원센터를 설치하고, 사회복지사 등의 권익 침해에 대한 상담과 지원, 권익 보호를 위한 교육 및 홍보, 권익 침해 실태와 권익 보호에 관한 조사 및 연구, 그 밖에 사회복지사 등의 권익 보호를 위한 �
